# Many Variables  GHZ

In [28]:
print('Started!')
from system import *
s1 = system('2',MMA=True,ManyVariables = True,TwoPhotonResonance=False)
#s1 = load_object('o-x-o')

Started!
Initializing system 2  ...
Constructing states and excitations...
Constructing ground and first-excited statespace...
Constructing gs_hamiltonian ...
Constructing e1_hamiltonian ...
Constructing interactions V_plus and V_minus ...
Constructing NJ_hamiltonian  ...
Inverting NJ_hamiltonian  ...
Constructing eff_hamiltonian and effective lindblau operators ...

System  2  initialized in 1.4 seconds.


# Effective Hamiltonian Matrix obtain 

In [32]:
#phi_val = np.pi/180 * 30
n = 0

total_indices = s1.variable_index

var('dct',domain='positive',  latex_name =r'\tilde{\delta }_c')
var('De1', domain='positive')
var('Deg1', domain='positive' , latex_name =r'{{\Delta e}}_{{  1}}^\gamma')
var('De2', domain='positive')
var('Deg2', domain='positive' , latex_name =r'{{\Delta e}}_{{  2}}^\gamma')
var('De3', domain='positive')
var('Deg3', domain='positive' , latex_name =r'{{\Delta e}}_{{  3}}^\gamma')    

var('gamma','DE','De','g','g_f','Omega','v','gamma_f','gamma_g','phi',domain='positive')
var('DEg', domain='positive' , latex_name =r'{{\Delta E}}^\gamma')
var('kappa_c','kappa_b',domain='positive')
var('C',domain='positive')
var('c',domain='positive',  latex_name =r'c')
var('r_f',domain='positive',latex_name =r'r_f')
var('r_g',domain='positive',latex_name =r'r_g')
var('R_f',domain='positive')#ratio  (g_f/g)^2
eff_hamiltonian_tayl = sg.copy( s1.eff_hamiltonian.parent().zero())
for diag in s1.pos_gs:
    print(f'Calculating {diag} out of {s1.pos_gs}...')
    min_expr_len = 10**10
    for term in range(1):
        b = s1.eff_hamiltonian[diag,diag]
        b = b.operands()[1].operands()[term]
        if len(str(b))<10: #we isolated Omega^2 instead of one of the conjugate pairs
            b = s1.eff_hamiltonian[diag,diag].operands()[0].operands()[term]
        
        b = b._mathematica_().ReplaceAll(f'DE->DEg*gamma*C^({n})')._sage_()
        #b = b.subs(DE = DEg*gamma*C^n)
        
        b = b._mathematica_().ReplaceAll(f'De1->Deg1*gamma*C^({n})')._sage_()
        #b = b.subs(De1 = Deg1*gamma*C^n) #sqrt C scaling!?
        
        b = b._mathematica_().ReplaceAll(f'De2->Deg2*gamma*C^({n})')._sage_()
        #b = b.subs(De2 = Deg2*gamma*C^n) #MAKE ZERO
        
        b = b._mathematica_().ReplaceAll(f'De3->Deg2*gamma*C^({n})')._sage_()
        #b = b.subs(De3 = Deg3*gamma*C^n) #MAKE ZERO  
        
        #b = b.subs(phi = phi_val)
        
        #b = MMA_simplify(b)
        
        #b = b._mathematica_().ReplaceAll('g_f->g*(R_f)^(1/2)')._sage_()
        b = b.subs(g_f=g*(R_f)^(1/2))

        b = b._mathematica_().ReplaceAll('gamma_f->gamma*r_f')._sage_()
        #b = b.subs(gamma_f=gamma*r_f)

        b = b._mathematica_().ReplaceAll('gamma_g->gamma*r_g')._sage_()
        #b = b.subs(gamma_g=gamma*r_g)
        
        #b = b._mathematica_().ReplaceAll("g-> (C*gamma*kappa_c)^(1/2)")._sage_()
        b = b.subs(g = (C*gamma*kappa_c)^(1/2)) 
        
        b = b._mathematica_().ReplaceAll('v->(c*kappa_b*kappa_c)^(1/2)')._sage_()
        
        b = b.subs(dc = (dct+0.5*I)*kappa_c )
        #b = b.subs(v = (c*kappa_b*kappa_c)^(1/2) )
        
        

        #Substitutions
        #b = b.subs(c=np.pi,R_f=2.1,Deg=np.e,DEg=np.sqrt(5),r_g=np.sqrt(3),r_f=np.sqrt(2), phi=phi_val) 
        #b = b.subs(phi=phi_val)

        #b = b._mathematica_().Factor()._sage_()
        #b = b._mathematica_().Together()._sage_()


        b = SR(str(b).replace('Sqrt','sqrt'))
        if len(str(b))<min_expr_len:
            min_expr_len = len(str(b))
            h_eff = b
    
    h_eff = h_eff.subs(c=C)
    h_eff = MMA_simplify(h_eff)
    '''
    h_eff = SR(str(h_eff).replace('Sqrt','sqrt'))
    var('x')
    h_eff = h_eff.subs(C=1/x)
    h_eff = MMA_simplify(h_eff)
    h_eff = h_eff._mathematica_().Series((x,0,1)).Normal()._sage_()
    h_eff = h_eff.subs(x=1/C)
    '''
    h_eff = SR(str(h_eff).replace('Sqrt','sqrt'))
    eff_hamiltonian_tayl[diag,diag] = h_eff  * Omega^2 / 2

print('\n Done! ')
#show(symround(eff_hamiltonian_tayl[s1.pos_gs,s1.pos_gs],digits=3))


eff_hamiltonian_tayl_temp = eff_hamiltonian_tayl[s1.pos_gs,s1.pos_gs] 





lind_op_number = len(s1.lindblau_list)

eff_lind_taylor = []
eff_lind = []
eff_lind_coeff = []
for lind_op in range(lind_op_number):
    print(f'{lind_op} out of {lind_op_number-1}')
    eff_lind_taylor.append([])
    eff_lind.append([])
    L_matrix = s1.eff_lindblau_list[lind_op]
    L_nonzeros = []
    L_nonzeros_pos = []
    for i in  range(L_matrix.nrows()):
        for j in  range(L_matrix.ncols()):
            if not str(symround(L_matrix[i,j]))==str(0):
                L_nonzeros.append(L_matrix[i,j])
                L_nonzeros_pos.append((i,j))
    eff_lind_coeff.append(s1.L_coeffs[lind_op])
    
    for which in range(len(L_nonzeros)):

        L_elem = L_nonzeros[which]
        b = L_elem
        
        b = b._mathematica_().ReplaceAll(f'DE->DEg*gamma*C^({n})')._sage_()
        #b = b.subs(DE = DEg*gamma*C^n)
        
        b = b._mathematica_().ReplaceAll(f'De1->Deg1*gamma*C^({n})')._sage_()
        #b = b.subs(De1 = Deg1*gamma*C^n) #sqrt C scaling!?
        
        b = b._mathematica_().ReplaceAll(f'De2->Deg2*gamma*C^({n})')._sage_()
        #b = b.subs(De2 = Deg2*gamma*C^n) #MAKE ZERO
        
        b = b._mathematica_().ReplaceAll(f'De3->Deg2*gamma*C^({n})')._sage_()
        #b = b.subs(De3 = Deg3*gamma*C^n) #MAKE ZERO  
        
        #b = b.subs(phi = phi_val)
        
        #b = MMA_simplify(b)
        
        #b = b._mathematica_().ReplaceAll('g_f->g*(R_f)^(1/2)')._sage_()
        b = b.subs(g_f=g*(R_f)^(1/2))

        b = b._mathematica_().ReplaceAll('gamma_f->gamma*r_f')._sage_()
        #b = b.subs(gamma_f=gamma*r_f)

        b = b._mathematica_().ReplaceAll('gamma_g->gamma*r_g')._sage_()
        #b = b.subs(gamma_g=gamma*r_g)
        
        #b = b._mathematica_().ReplaceAll("g-> (C*gamma*kappa_c)^(1/2)")._sage_()
        b = b.subs(g = (C*gamma*kappa_c)^(1/2)) 
        
        b = b._mathematica_().ReplaceAll('v->(c*kappa_b*kappa_c)^(1/2)')._sage_()
        b = b.subs(dc = (dct+0.5*I)*kappa_c )
        b = b.subs(c=C)
        
        b = SR(str(b).replace('Sqrt','sqrt'))
        
        eff_lind[lind_op].append(b)
        
print('\n Done!')


eff_hamiltonian_tayl = eff_hamiltonian_tayl_temp

a = eff_hamiltonian_tayl[0,0]
#a = symround( MMA_simplify(a,full=False) ,digits = 14)
b = eff_hamiltonian_tayl[1,1]
#b = symround(MMA_simplify(b,full=False),digits = 14)
c = eff_hamiltonian_tayl[3,3]
#c = symround(MMA_simplify(c,full=False),digits = 14)
d = eff_hamiltonian_tayl[4,4]
#d = symround(MMA_simplify(d,full=False),digits = 14)
e = eff_hamiltonian_tayl[5,5]
#e = symround(MMA_simplify(e,full=False),digits = 14)
f = eff_hamiltonian_tayl[7,7]
#f = symround(MMA_simplify(f,full=False),digits = 14)

Calculating 0 out of [0, 1, 2, 3]...
Calculating 1 out of [0, 1, 2, 3]...
Calculating 2 out of [0, 1, 2, 3]...
Calculating 3 out of [0, 1, 2, 3]...

 Done! 
0 out of 4
1 out of 4
2 out of 4
3 out of 4
4 out of 4

 Done!


IndexError: matrix index out of range

# See results

In [9]:
m = 1/2

In [16]:
x = symround(d.subs(R_f=2.1,Deg1=np.sqrt(2)*C^m,Deg2=np.e*C^m,DEg=np.sqrt(5)*C^m,r_g=np.sqrt(3),gamma=0.3,r_f=0.1*np.sqrt(2), phi=np.pi/10,gamma_f=1.45,gamma_g=3),digits=14)
x = x._mathematica_().Together().sage()
print('Numerator')
show(symround(x._mathematica_().Numerator()._sage_().expand(),digits=5))
print('\n\nDenominator')
show(symround(x._mathematica_().Denominator()._sage_().expand(),digits=5))

NameError: name 'd' is not defined

In [248]:
#m = 3

lindop = 7
pos = 7

err = eff_lind[lindop][pos].subs(Omega=1,kappa_c=5.61,kappa_b=1.79, R_f=2.1,Deg1=np.sqrt(2)*C^m,Deg2=np.e*C^m,DEg=np.sqrt(5)*C^m,r_g=np.sqrt(3),gamma=0.3,r_f=0.1*np.sqrt(2), phi=np.pi/10,gamma_f=1.45,gamma_g=3)

err = err._mathematica_().Together()._sage_() 
err = SR(str(( err)).replace('Sqrt','sqrt'))
#err = SR(str(MMA_simplify( err,full=False )).replace('Sqrt','sqrt')).simplify()
#show( symround( err ,digits=1) )
print(f'coeff:')
show(eff_lind_coeff[lindop])

coeff:


sqrt(gamma_g)

In [249]:
el = 0
q = (symround(err._mathematica_().Numerator()._sage_() ,tol=1e-30 ).expand().operands()[el])

print('Numerator')

show(symround(q._mathematica_().Numerator()._sage_(),digits=3).expand())

print(obtain_exponent(symround(q._mathematica_().Numerator()._sage_(),digits=3).expand()))

print('\n\nDenominator')
show(symround(q._mathematica_().Denominator()._sage_(),digits=3).expand())
print(obtain_exponent(symround(q._mathematica_().Denominator()._sage_(),digits=3).expand()))

print('\n\nextra Denominator')
show( symround(err._mathematica_().Denominator()._sage_().expand(),digits=3))

print(obtain_exponent(symround(err._mathematica_().Denominator()._sage_().expand(),digits=3)))

Numerator


AttributeError: 'list' object has no attribute '_mathematica_'

In [234]:
show(symround(err._mathematica_().Numerator()._sage_().expand(),tol=1e-30))
#show(symround( err.numerator().expand()))

-(22. - 7.2*I)*C^12 + (60. + 190.*I)*C^12. - (162*I - 496)*C^11 - (420*I + 152)*C^10. + (14. + 6.8*I)*C^10 - (56. + 180.*I)*C^9.5 - (60. - 20.*I)*C^9 - (6.1 + 20.*I)*C^8.5 - (14. - 4.4*I)*C^8 + (0.23 + 0.72*I)*C^7.5 - (0.92 - 0.30*I)*C^7 + (0.063 + 0.20*I)*C^6.5 - (0.0073 - 0.0023*I)*C^6 + (0.0022 + 0.0067*I)*C^5.5 + (0.00082 - 0.00026*I)*C^5 - (0.000010 + 0.000032*I)*C^4.5 + (7.7e-21 + 1.5e-20*I)*C^4 + (4.4e-22 - 1.4e-21*I)*C^3.5

In [235]:
#show(symround(err.denominator().expand()))
den = err._mathematica_().Denominator()._sage_().expand()
den = SR(str(MMA_simplify( den,full=False )).replace('Sqrt','sqrt')).simplify()
show(symround(den,tol=1e-30))

(4.4 + 15.*I)*C^13 - (63*I - 226)*C^12. - (1216*I + 328)*C^12 + (720*I - 2720)*C^12. + (1536*I + 420)*C^11 + (624*I - 2976)*C^10. + (3776*I + 808)*C^10 - (166*I - 214)*C^9.5 + (1456*I + 142)*C^9 - (71*I - 504)*C^8.5 - (0.66 - 180.*I)*C^8 + (120. - 5.7*I)*C^7.5 - (1.1 - 0.55*I)*C^7 + (11. - 0.052*I)*C^6.5 - (0.040 + 1.6*I)*C^6 + (0.39 + 0.0046*I)*C^5.5 + (0.00018 - 0.12*I)*C^5 - (0.0026 - 6.9e-18*I)*C^4.5 - (1.2e-18 + 0.0023*I)*C^4 - (0.00031 - 4.2e-19*I)*C^3.5 + (5.3e-20 + 0.000012*I)*C^3 - (1.1e-21 + 5.8e-21*I)*C^2.5 - (3.0e-22 - 4.1e-22*I)*C^2

# Testing for auto

In [33]:
def obtain_exponent(expr):
    expr = SR(str(( expr)).replace('Sqrt','sqrt'))
    try:
        exp = expr.op[0].op[0].op[0].op[1]
    except:
        try:
            exp = expr.op[0].op[0].op[1]
        except:
            try:
                exp = expr.op[0].op[1]
            except:
                exp = 0
    return exp

def obtain_expr_exp(expr):
    try:
        el = 0    
        q = (expr._mathematica_().Numerator()._sage_() .expand().operands()[el])
        q = SR(str(( q)).replace('Sqrt','sqrt'))
        test_bool = (str(q._mathematica_().Numerator()._sage_().expand().denominator()) == str(1))
        num_exp = (obtain_exponent(q._mathematica_().Numerator()._sage_().expand()))
        den1_exp = (obtain_exponent(q._mathematica_().Denominator()._sage_().expand()))
        den2_exp = (obtain_exponent(expr._mathematica_().Denominator()._sage_().expand()))
        res = num_exp-den1_exp-den2_exp
        if not test_bool:
            #show(q)
            res = obtain_exponent(q.numerator())-obtain_exponent(q.denominator())
            return res
        return res
    except:
        #print('altern')
        den_exp = obtain_exponent( expr._mathematica_().Denominator()._sage_().expand())
        num_exp = obtain_exponent( expr._mathematica_().Numerator()._sage_().expand())
        return num_exp-den_exp


In [34]:
mx = -100   #maximum value of exponent
m = 1       #exponent of detunings

phival = 0* np.pi 
R_fval = 1
gamma_fval = gamma
gamma_gval = gamma
DEgval = 1
Deg2val = DEgval*minsolD2  #*0.99999
Deg1val = DEgval*minsolD1

for lindop in range(13):
    show(lindop,eff_lind_coeff[lindop]  )
    pos = 0
    more = True
    while(more):
        try: 
            err = eff_lind[lindop][pos].subs(dct=np.sqrt(1.53)*C^m,Omega=1,kappa_c=5.61,kappa_b=1.79, R_f=2.1,Deg1=np.sqrt(2)*C^m,Deg2=np.e*C^m,DEg=np.sqrt(5)*C^m,r_g=np.sqrt(3),gamma=0.3,r_f=0.1*np.sqrt(2), phi=np.pi/10,gamma_f=1.45,gamma_g=3)
            err = err._mathematica_().Together()._sage_() 
            exp = obtain_expr_exp(err)
            if exp > mx:
                mx = exp
            pos += 1
            print(exp)
        except IndexError:
            more = False

0 sqrt(kappa_c)

-3/2
-5/2
-5/2
-3/2


1 sqrt(gamma_g)

-1.00000000000000
-2
-2
-1


2 sqrt(gamma_f)

...

3 sqrt(gamma)

-2
-3


4 sqrt(gamma)

-2
-3


IndexError: list index out of range

In [26]:
show(mx)

-1

In [27]:
show(expr)

NameError: name 'expr' is not defined

7/2

In [188]:
#TEST
def obtain_exponent(expr):
    try:
        orde = SR(str(( expr)).replace('Sqrt','sqrt')).Order()
        exp = orde.op[0].op[0].op[0].op[1]
    except TypeError:
        exp = 0
    return exp

orde.op[0].op[0].op[0].op[1]

In [ ]:
el = 0
q = (symround(err._mathematica_().Numerator()._sage_() ,tol=1e-30 ).expand().operands()[el])

print('Numerator')

show(symround(q._mathematica_().Numerator()._sage_(),digits=3).expand())

print(obtain_exponent(symround(q._mathematica_().Numerator()._sage_(),digits=3).expand()))

print('\n\nDenominator')
show(symround(q._mathematica_().Denominator()._sage_(),digits=3).expand())
print(obtain_exponent(symround(q._mathematica_().Denominator()._sage_(),digits=3).expand()))

print('\n\nextra Denominator')
show( symround(err._mathematica_().Denominator()._sage_().expand(),digits=3))

print(obtain_exponent(symround(err._mathematica_().Denominator()._sage_().expand(),digits=3)))

In [250]:
show(err)

(-(7488.95904084581 + 6718.01634154288*I)*C^19.5000000000000 - (146175.167483481 - 158210.139494543*I)*C^19.0000000000000 + (1.40416170190662e6 + 1.33575762375496e6*I)*C^18.5000000000000 + (6.55178140330077e6 - 6.69860005031530e6*I)*C^18.0000000000000 - (1.79305037473182e7 + 1.79350587849144e7*I)*C^17.5000000000000 - (2.36155772108600e7 - 2.37171435868918e7*I)*C^17.0000000000000 + (1.63910100882983e6 - 1.84406936185372e6*I)*C^16.5000000000000 - (4.75285059889809e7 - 3.75433061064312e7*I)*C^16.0000000000000 + (4.15313038066807e7 + 4.87626257153558e7*I)*C^15.5000000000000 - (6.33440670519635e6 + 3.75542957225609e6*I)*C^15.0000000000000 + (2.10321093375663e7 + 3.96699134023395e7*I)*C^14.5000000000000 + (1.70216570785738e7 - 1.18298309937309e7*I)*C^14.0000000000000 + (2.22992515803455e6 + 1.02781952723612e7*I)*C^13.5000000000000 + (9.81230665729459e6 - 3.90679892890652e6*I)*C^13.0000000000000 - (696896.795467959 + 96640.1962038990*I)*C^12.5000000000000 + (2.33199366790597e6 - 496680.108507839*I)*C^12.0000000000000 - (219610.749410932 + 593575.543038401*I)*C^11.5000000000000 + (261158.966150310 - 10810.5896413446*I)*C^11.0000000000000 - (23858.2442671281 + 137588.872226217*I)*C^10.5000000000000 + (5835.04119434564 + 3623.60234960061*I)*C^10.0000000000000 - (981.808846066121 + 15187.0283761388*I)*C^9.50000000000000 - (1974.40536022792 - 369.849243882553*I)*C^9.00000000000000 + (13.5466895709047 - 830.287328943550*I)*C^8.50000000000000 - (248.206992393050 - 12.0218190295012*I)*C^8.00000000000000 + (2.19390691746541 - 9.76057021050649*I)*C^7.50000000000000 - (12.6508870551941 + 0.0180564966983194*I)*C^7.00000000000000 + (0.0392953502101259 + 1.29303548235977*I)*C^6.50000000000000 - (0.236676913721802 + 0.00554130087105360*I)*C^6.00000000000000 - (0.000332856535077300 - 0.0632165983131768*I)*C^5.50000000000000 + (0.00273672965613837 + 7.85190215809965e-6*I)*C^5.00000000000000 + (2.99122174025831e-20 + 0.000714595972831589*I)*C^4.50000000000000 + (0.000117847368191749 - 1.27131118366605e-19*I)*C^4.00000000000000 - (2.29102610635849e-20 + 7.29137651509710e-6*I)*C^3.50000000000000 - (1.71999552242979e-7 - 2.45775303411946e-21*I)*C^3.00000000000000 + (1.56015418193604e-22 - 1.15734856339218e-22*I)*C^2.50000000000000 - (5.88477778225279e-24 + 4.27554132666771e-24*I)*C^2)/((1.00000000000000*C^10.0000000000000 - (2.36605818352689e-15 + 10.9863242408896*I)*C^9.50000000000000 - (36.5687902975831 - 1.30041145233382e-14*I)*C^9.00000000000000 + (1.43986842710431e-14 + 30.7526943011312*I)*C^8.50000000000000 - (33.4919503889781 - 3.54829052614286e-15*I)*C^8.00000000000000 + (1.19061331248695e-14 + 45.7451757962172*I)*C^7.50000000000000 + (5.66435160775730 - 2.55444014160724e-15*I)*C^7.00000000000000 + (1.88813131558382e-15 + 9.46282259996348*I)*C^6.50000000000000 + (3.10462986284868 - 1.88079758141438e-16*I)*C^6.00000000000000 - (9.43327633504782e-17 - 0.413204334079750*I)*C^5.50000000000000 + (0.295430815711153 + 3.08708169387755e-17*I)*C^5.00000000000000 - (2.55053762809913e-18 + 0.0226004009812007*I)*C^4.50000000000000 + (0.00696967726456589 - 2.63231827019972e-18*I)*C^4.00000000000000 - (1.83054627913405e-18 + 0.00135071328592904*I)*C^3.50000000000000 - (0.0000684213696255791 - 2.04267216462088e-19*I)*C^3.00000000000000 + (2.35383169358941e-20 + 6.98522960528717e-21*I)*C^2.50000000000000 - (2.34096281387470e-21 + 1.70080904075907e-21*I)*C^2)*((4.34672582434281 + 15.1235908447054*I)*C^3.00000000000000 + (58.9179361743329 - 15.2169042607225*I)*C^2.50000000000000 - (2.65192677524963 + 11.4175364591636*I)*C^2 + (16.7467080330417 - 5.86014199342403e-16*I)*C^1.50000000000000 - (1.25420899446302e-16 + 3.58419162304540*I)*C - (7.50066723502131e-18 - 0.285651125213588*I)*C^1.00000000000000 + 1.00000000000000*sqrt(C) - 1.48465718859219e-17 - 0.174275051601312*I)*((4.34672582434281 + 15.1235908447054*I)*C^3.00000000000000 + (58.9179361743329 - 15.2169042607225*I)*C^2.50000000000000 - (2.65192677524963 + 11.4175364591636*I)*C^2 + (16.7467080330417 + 9.38354